1. Familiarize basic python packages for deep learning such as Keras, Tensorflow etc.
2. Data pre-processing operations such as outliers and/or inconsistent data value
management. **
3. Implement Feed forward neural network with three hidden layers for classification
on CIFAR-10 dataset.**
4. Analyse the impact of optimization and weight initialization techniques such as
Xavier initialization, Kaiming Initialization, dropout and regularization techniques
and visualize the change in performance. **
5. Digit classification using CNN architecture for MNIST dataset. **
6. Digit classification using pre-trained networks like VGGnet-19 for MNIST dataset
and analyse and visualize performance improvement.**
7. Implement a simple RNN for review classification using IMDB dataset.**
8. Analyse and visualize the performance change while using LSTM and GRU
instead of simple RNN.**
9. Implement time series forecasting prediction for NIFTY-50 dataset. **
10. Implement a shallow auto encoder and decoder network for machine translation(by
using Kaggle English to Hindi neural translation dataset

left - 3 4 7

done - 2 5 8 9 6

Program 8,9

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Flatten, LSTM, GRU
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator

sunspots = pd.read_csv('/content/Sunspots.csv', index_col='Date', parse_dates=True)
sc = MinMaxScaler(feature_range = (0,1))
data = sc.fit_transform(sunspots).flatten()
data.shape
train = data[:5330]
test = data[5330:]


n_input = 7
n_features = 1

timetrain = TimeseriesGenerator(train, train, length=n_input, batch_size=1 )
timetest = TimeseriesGenerator(test, test, length=n_input, batch_size=1 )

model = Sequential()
model.add(GRU(units=100, input_shape=(n_input,n_features) , activation='relu',))
# model.add(LSTM(units=100, input_shape=(n_input,n_features) , activation='relu',))
# model.add(SimpleRNN(units=100, input_shape=(n_input,n_features) , activation='relu',))
model.add(Dense(units=1, activation='relu'))

model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(timetrain, epochs=5)
predict = model.predict(timetest)

Program 5

In [ ]:
#Using MNIST dataset
import tensorflow as tf
from keras.datasets import mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()
len(set(y_test.flatten()))

from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train, num_classes = 10)
y_test = to_categorical(y_test, num_classes = 10)
#Preprocessing
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

#Normalize pixel values to the range [0,1] (pixel values range from 0 to 255)
X_train /= 255
X_test /= 255

#Finding input size
X_train.shape

from keras.models import Sequential
model = Sequential()

from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
#Block1
model.add(Conv2D(filters=25, kernel_size=(3,3), padding='same', activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D((2,2)))
#Block2
model.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu'))
model.add(MaxPooling2D((2,2)))
#block3
model.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu'))
model.add(MaxPooling2D((2,2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))


#categorical_crossentropy is used for multi class classification tasks
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()
history = model.fit(X_train, y_train, epochs=1, validation_data=(X_test, y_test))

In [ ]:
import tensorflow as tf
from tensorflow.keras import models, layers
from keras.datasets import cifar10
import numpy as np
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

(X_train, y_train), (X_test, y_test) = cifar10.load_data()

y_train = to_categorical(y_train, num_classes = 10)
y_test = to_categorical(y_test, num_classes = 10)
y_train
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= 255
X_test /= 255
# X_train

model = Sequential()
model.add(Conv2D(filters = 25, kernel_size = (3,3), padding = 'same', activation = 'relu', input_shape = (32, 32, 3)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = 'same', activation = 'relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = 'same', activation = 'relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())
model.add(Dense(128, activation = 'relu'))
model.add(Dense(10, activation = 'softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

history = model.fit(X_train, y_train, epochs = 1, validation_data = (X_test, y_test))
score = model.evaluate(X_test, y_test, batch_size = 128, verbose = 0)
print(score)


Program 6

In [ ]:
import tensorflow_datasets as tfds

import tensorflow as tf
from keras.utils import to_categorical

from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
from keras.callbacks import EarlyStopping

(X_train, y_train), (X_test,y_test) = tfds.load('tf_flowers', split=["train[:70%]","train[30%:]"], batch_size=-1, as_supervised=True)


X_train = tf.image.resize(X_train,size=(150,150))
X_test = tf.image.resize(X_test,size=(150,150))

y_train = to_categorical(y_train, num_classes=5)
y_test = to_categorical(y_test, num_classes=5)

base_model = VGG16(weights='imagenet',include_top=False,input_shape=X_train[0].shape)
base_model.trainable = False

model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(units=50, activation='relu'))
model.add(Dense(units=20, activation='relu'))
model.add(Dense(units=5, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

es = EarlyStopping(monitor="val_loss", mode="max", patience=5, restore_best_weights=True)
history = model.fit(X_train, y_train, validation_split=0.2, batch_size=32, epochs=1, callbacks=[es])
test_pred=model.predict(X_test)
score= model.evaluate(X_test,y_test, batch_size=128, verbose=0)
print(score)


Program 7


In [ ]:
# https://towardsdatascience.com/a-beginners-guide-on-sentiment-analysis-with-rnn-9e100627c02e
from keras.datasets import imdb

vocabulary_size = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = vocabulary_size)

print('Loaded dataset with {} training samples, {} test samples'.format(len(X_train), len(X_test)))
print('---review---', X_train[6])
print('---label---', y_train[6])

word2id = imdb.get_word_index()
id2word = {i: word for word, i in word2id.items()}
print('---review with words---',[id2word.get(i, ' ') for i in X_train[6]])
print('---label---', y_train[6])
print('Maximum review length: {}'.format(len(max((X_train + X_test), key=len))))
print('Minimum review length: {}'.format(len(min((X_test + X_test), key=len))))

from keras.utils import pad_sequences
max_words = 500
X_train = pad_sequences(X_train, maxlen=max_words)
X_test = pad_sequences(X_test, maxlen=max_words)

from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
embedding_size=32
model=Sequential()
model.add(Embedding(vocabulary_size, embedding_size, input_length=max_words))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

print(model.summary())
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

batch_size = 64
num_epochs = 3

X_valid, y_valid = X_train[:batch_size], y_train[:batch_size]
X_train2, y_train2 = X_train[batch_size:], y_train[batch_size:]
model.fit(X_train2, y_train2, validation_data=(X_valid, y_valid), batch_size=batch_size, epochs=num_epochs)
scores = model.evaluate(X_test, y_test, verbose=0)

print('Test accuracy:', scores[1])

K=X_test
K1=X_test[1]
model.predict(K1)
X_train2[0]


In [ ]:
import pandas as pd

data = pd.read_csv("IMDB_Dataset.csv")

data.head()

print(data.review[0])

import re


# html =
# tags = r'[/<(?:"[^"]*"['"]*|'[^']*'['"]*|[^'">])+>/g;]'
print(re.sub("<[^>]*>", ' ', data.review[0]))
tags_removed = re.sub("<[^>]*>", ' ', data.review[0])

html = r'[^0-9A-Za-z\s]'
print(re.sub(html, ' ', tags_removed))
alpha = re.sub(html, ' ', tags_removed)

pip install nltk

from nltk.tokenize.toktok import ToktokTokenizer

tokenizer = ToktokTokenizer()
tokens = tokenizer.tokenize(alpha)
print(tokens)

from nltk.stem import PorterStemmer

stemmer = PorterStemmer()
stemWords = [stemmer.stem(word) for word in tokens]
print(stemWords)

import nltk
stopwords = nltk.corpus.stopwords.words('english')
print(stopwords)
print(len(stopwords))


nltk.download("stopwords")

filtered_words = []
for token in stemWords:
    if token not in stopwords:
        filtered_words.append(token)
print(filtered_words)
print(len(filtered_words))

visualization

In [ ]:
def plot_result(trainY, testY, train_predict, test_predict):
    actual = np.append(trainY, testY)
    predictions = np.append(train_predict, test_predict)
    rows = len(actual)
    plt.figure(figsize=(15, 6), dpi=80)
    plt.plot(range(rows), actual)
    plt.plot(range(rows), predictions)
    plt.axvline(x=len(trainY), color='r')
    plt.legend(['Actual', 'Predictions'])
    plt.xlabel('Observation number after given time steps')
    plt.ylabel('Sunspots scaled')
    plt.title('Actual and Predicted Values. The Red Line Separates The Training And Test Examples')
plot_result(trainY, testY, train_predict, test_predict)